# ECON 326 Group Final Report
#### Group 14 - Bhavya Dubey, Kashie Ugoji, Ruiquin Wang, Zhuoying Sun

## <u>Introduction<u>

When looking at the current nature of the financial markets, it is clear how much influence they have in our society. With around 63% of young adults (ages 18-34) **(remember to cite)** believing in building their wealth through the stock exchange, it shows how much of a focal point this aspect of our economy is for many people. 

## <u>Data Description<u>

## <u>Summary Statistics<u> 

## <u>Model Specification<u>

## <u>Table of Results<u>

## <u>Discussion<u>

## <u>Specification Check<u>

## <u>Robustness Analysis<u>

## <u>Conclusion<u>

## <u>References<u>